In [4]:
import pandas as pd
import nltk
nltk.download("stopwords")
#--------#

import multiprocessing as mlp
from multiprocessing import Pool
from tqdm import tqdm
 
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
import numpy as np

[nltk_data] Downloading package stopwords to C:\Users\Xiaomi
[nltk_data]     Pro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
df1 = pd.read_csv('./../resume_to_hackaton.csv', delimiter=";")

In [6]:
df1

,uuid,position,organization,description,start,end
0,e08588cc-4ad1-45be-ae95-fa3fd31bc32e,заведующая кафе-кладовщик,ООО Мираж,Составление ттк\r Проведение инвентаризаций\r ...,2012-04,2012-11
1,36c13102-4a62-477b-a475-f9a5b1c48218,Заведующий складом,Оконный завод,руководство складом и цехом по производству ко...,2014-02,2015-09
2,36c13102-4a62-477b-a475-f9a5b1c48218,старший кладовщик,"ООО "" Керамика сервис""",Полное ведение складского учета отдельно стоящ...,2012-03,2013-06
3,36c13102-4a62-477b-a475-f9a5b1c48218,бухгалтер складского учета,"Многопрофильный холдинг ""Вероника"" СПБ","Cклад, бухгалтер по учету ТМЦ и ОС, проведение...",2010-12,2011-05
4,85bfe532-1108-4142-bae5-29818bfc70a0,Старший официант,"ДВЕ ПАЛОЧКИ, сеть ресторанов свободного полета",•\tПодготовка ресторана к открытию;\r •\tВстре...,2008-12,2011-02
...,...,...,...,...,...,...
99995,df2f6158-c03b-4545-a4c5-e5bf80fd5a5b,Администратор ресторана,"Евразия, сеть ресторанов и суши-баров",Контроль работы ресторана,2014-06,2014-12
99996,df2f6158-c03b-4545-a4c5-e5bf80fd5a5b,Официант,jazz bar 48 chairs,Обслуживание гостей по всем европейским станда...,2011-06,2014-06
99997,df2f6158-c03b-4545-a4c5-e5bf80fd5a5b,"Суши-повар, официант, администратор",ООО Страйк,На поседней должности-контроль работы ресторана.,2007-04,2011-05
99998,7b0b9de2-d04c-42d5-a59d-6808b5abef8a,Член бригады ресторана,Макдоналдс,"Кассир, повар, грузчик.",2013-10,2014-11


In [4]:
df2 = pd.read_csv('./../vacancies_to_hackaton.csv', delimiter=";")

In [5]:
df2

,id,name,area.name,city,company.id,company,company_link,publication_date,salary_from,salary_currency,employment.name,schedule.name,experience.name,key_skills,specializations,description
0,17016760,Официант,Санкт-Петербург,Санкт-Петербург,1905558,2Т,https://hh.ru/employer/1905558,2016-06-16T11:22:58+0300,30000.0,RUR,Полная занятость,Полный день,Нет опыта,NaN,22.193 15.313,"Мы ищем активного, жизнерадостного и дружелюбн..."
1,17759717,Бармен-официант,Санкт-Петербург,unknown,2440865,3Ф Групп,https://hh.ru/employer/2440865,2016-07-15T13:00:14+0300,30000.0,RUR,Полная занятость,Полный день,Нет опыта,Ответственность | Материальная ответственность...,22.175 22.193,Обязанности: Приготовление напитков в баре ...
2,17816582,Администратор ресторана,Санкт-Петербург,Санкт-Петербург,1911500,4ЕС,https://hh.ru/employer/1911500,2016-07-20T17:20:48+0300,45000.0,RUR,Полная занятость,Сменный график,От 3 до 6 лет,NaN,22.329,"Ресторан ""Рандеву"" приглашает на работу админи..."
3,17746005,Кладовщик,Санкт-Петербург,посёлок Шушары,141100,4Ф,https://hh.ru/employer/141100,2016-08-13T09:26:06+0300,38000.0,RUR,Полная занятость,Сменный график,От 1 года до 3 лет,NaN,21.275 21.563,Обязанности: Прием товара (Кока-Кола) на скла...
4,17431972,Кладовщик,Санкт-Петербург,поселок Шушары,141100,4Ф,https://hh.ru/employer/141100,2016-07-12T15:22:23+0300,30000.0,RUR,Полная занятость,Полный день,От 1 года до 3 лет,NaN,21.563 21.564,Обязанности: отпуск и прием на складе ТМЦ вед...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32160,26344256,Кладовщик,Санкт-Петербург,Санкт-Петербург,3414,"ТрансМашХолдинг, Группа компаний",https://hh.ru/employer/3414,2018-06-13T19:25:26+0300,NaN,NaN,Полная занятость,Полный день,От 1 года до 3 лет,NaN,21.563,"ОАО ""Октябрьский электровагоноремонтный завод""..."
32161,26367364,Кондитер в ресторан (частичная занятость),Санкт-Петербург,unknown,3064104,ТРИГГЕР,https://hh.ru/employer/3064104,2018-06-14T17:56:22+0300,29000.0,RUR,Частичная занятость,Сменный график,От 1 года до 3 лет,NaN,22.204 22.353,"Ресторан ""Счастье у Исаакия"" открывает конкурс..."
32162,26373024,Заведующий производством/заведующий продовольс...,Санкт-Петербург,Пушкин,2130686,ФГБУЗ Санкт-Петербургский Дом-пансионат ветера...,https://hh.ru/employer/2130686,2018-06-15T09:56:47+0300,30000.0,RUR,Полная занятость,Полный день,От 1 года до 3 лет,NaN,22.204 18.297,Обязанности: - Руководство деятельностью пищеб...
32163,26431613,Грузчик-комплектовщик,Санкт-Петербург,unknown,66239,"Элизиум, группа компаний",https://hh.ru/employer/66239,2018-06-19T10:45:48+0300,30000.0,RUR,Полная занятость,Полный день,Нет опыта,NaN,29.495 29.545 21.275 21.564 21.136,Обязанности: Перемещение готовой продукции По...


In [6]:

#Create lemmatizer and stopwords list
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")

#Preprocess function
def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [i.split(" ") for i in tokens]
    tokens = np.concatenate(tokens)
    tokens = [token.strip() for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation \
              and not token.strip().isdigit()]
    return tokens


In [7]:
desc = pd.read_excel("./../описание полей.xlsx", index_col=0, header=None)

In [8]:
m1 = {"q1": "Желаемая должность",
"q2": "У вас есть опыт работы кладовщиком?",
"q2d": "У вас есть опыт работы кладовщиком? строка",
"q3": "Укажите Ваш стаж работы кладовщиком",
"q4": "Отрасль",
"q5": "Названия компаний",
"q6": "С какой системой хранения Вы имели опыт работы?",
"q7": "Опыт работы с программами",
"q8": "Опыт с инструментарием",
"q9": "Типы работ",
"q10": "Сколько времени вы потратили на поиск работы кладовщика?",
"q11": "Уровень ЗП (желаемый)"
}

In [9]:
m2 = {"q1": "Желаемая должность",
"q2": "У вас есть водительские права?",
"q3": "Водительское удостоверение",
"q4": "Удостоверение тракториста-машиниста нового образца",
"q5": "Удостоверение тракториста-машиниста старого образца",
"q6": "У вас есть опыт работы водителем погрузчиком?",
"q7": "Ваш стаж работы водителем погрузчика",
"q8": "Сфера деятельности",
"q9": "Названия компаний",
"q10": "Какой техникой вы управляете?",
"q11": "Максимальная высота поднятия груза",
"q12": "С какой системой хранения Вы имели опыт работы?",
"q13": "удостоверение стропальщика?",
"q14": "Типы работ",
"q15": "Возможность провести тест-драйв на собеседовании",
"q16": "Ключевые качества",
"q17": "Знание языков",
"q19": "Хобби",
"q20": "Уровень ЗП",
"q21": "График работы",
"q22": "Ночная смена",
"q23": "Командировки",
"q24": "Станции метро"
}


In [10]:
m3 = {
"q1": "Желаемая должность",
"q2": "Есть ли у Вас профессиональное образование в сфере гостеприимства?",
"q3": "профессиональное образовательное учреждение, где вы учились, и специальность",
"q4": "Год окончания обучения",
"q5": "Стаж работы официантом",
"q6": "Рестораны, где работал",
"q7": "дополнительные профессиональные знания и навыки",
"q8": "Знание языков",
"q9": "C какой посудой работали",
"q10": "График работы"
}


In [ ]:
class BagOfWordCoder:
    vectorizer = None
    __init__(self, path_to_corpus):
        if not path_to_corpus.endswith(".vec"):
            raise Exception(f"Need to be *.vec file")
        with open(path_to_corpus) as f:
            vectorizer = pickle.load(f)
    
    def vectorize(self, text):
        return vectorizer.transform(text)
    
    def get_text(self, matrix):
        return le.inverse_transform(matrix)
    

In [ ]:
nltk.tokenize.load()